In [362]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re
import pandas as pd
import collections
import time

In [68]:
PATH = "C:\Program Files (x86)\chromedriver_win32\chromedriver.exe"
s=Service(PATH)
driver = webdriver.Chrome(service=s)
driver.get('https://www.fotmob.com/')
games = driver.find_elements(By.CLASS_NAME, 'ew7iiy60')
links = [game.get_attribute('href') for game in games]

In [357]:
time

<module 'time' (built-in)>

In [502]:
url = 'https://www.fotmob.com/match/3626027/matchfacts/marseille-vs-lille'
driver = webdriver.Chrome(service=s)
driver.get(url)
allstats = driver.find_elements(By.XPATH , '(//button[@class="css-1isl7sd-ExpandButton e7i6a0l1"])[2]')[0]
allstats.click()
time.sleep(2)
stats = driver.find_elements(By.CLASS_NAME, 'e360fsv1')[1].text
game = driver.find_elements(By.CLASS_NAME, 'css-jkaqxa')[1].text
times = driver.find_elements(By.CLASS_NAME, 'css-xj92le')[1].text
driver.quit()

In [503]:
stats

'11\nShots\n5\n5\nShots on target\n4\n4\nShots off target\n1\n2\nBlocked shots\n0\n0\nShots woodwork\n0\n7\nShots inside box\n2\n4\nShots outside box\n3\n0.92\nExpected goals (xG)\n0.75\n0.77\nxG first half\n0.75\n0.15\nxG second half\n0.00\n1.71\nxG on target (xGOT)\n1.31\n0.19\nxG open play\n0.68\n0.73\nxG set play\n0.06\nWhat is xG?\n34\nDuels won\n18\n52\nDuels\n52\n6\nDribbles attempted\n8\n4\nDribbles succeeded\n2\n13\nTackles attempted\n7\n6\nTackles succeeded\n3\n9\nAerials won\n3\n7\nInterceptions\n2\n358\nAccurate passes\n118\n131\nOwn half\n85\n227\nOpposition half\n33\n410\nPasses\n162\n87%\nPass success\n73%\n519\nTouches\n279\n22\nLong balls\n30\n17\nAccurate long balls\n8\n20\nCrosses\n4\n6\nAccurate crosses\n2\n13\nThrows\n14\nDiscipline\n1\nYellow cards\n2\n0\nRed cards\n1\nKeeper\n3\nSaves\n5\n2\nDiving saves\n1\n1\nSaves inside box\n4\n0\nActed as sweeper\n0\n0\nPunches\n1'

In [504]:
score = game.split()[::2]
goal1 = score[0]
goal2 = score[1]

In [505]:
los = re.split(pattern = "\\r?\\n", string = stats)
los.remove('What is xG?')
los.remove('Keeper')
los.remove('Discipline')


In [506]:
len(los)

117

In [484]:
columns = los[1::3]
columns.insert(0, 'Goal')
team1 = los[::3]
team1.insert(0, goal1)
team2 = los[2::3]
team2.insert(0, goal2)

In [496]:
len(columns)

39

In [495]:
list(row.keys())[0:40]

['1 Goal', '1 Shots', '1 Shots on target', '1 Shots off target', '1 Blocked shots', '1 Shots woodwork', '1 Shots inside box', '1 Shots outside box', '1 Expected goals (xG)', '1 xG first half', '1 xG second half', '1 xG on target (xGOT)', '1 xG open play', '1 xG set play', '1 Duels won', '1 Duels', '1 Dribbles attempted', '1 Dribbles succeeded', '1 Tackles attempted', '1 Tackles succeeded', '1 Aerials won', '1 Interceptions', '1 Accurate passes', '1 Own half', '1 Opposition half', '1 Passes', '1 Pass success', '1 Touches', '1 Long balls', '1 Accurate long balls', '1 Crosses', '1 Accurate crosses', '1 Throws', '1 Yellow cards', '1 Red cards', '1 Saves', '1 Diving saves', '1 Saves inside box', '1 Acted as sweeper', '1 Punches']

In [236]:
dfone = pd.DataFrame([team1], columns = columns).add_prefix('1 ')
dftwo = pd.DataFrame([team2], columns = columns).add_prefix('2 ')
df = dfone.join(dftwo)

In [455]:
cols = ['1 ' + word for word in columns] + ['2 ' + word for word in columns]
gs = team1 + team2
row = {k:v for k, v in zip(cols, gs)}

In [457]:
fd = collections.defaultdict(list)

for d in [row]:
    for key, value in d.items():
        fd[key].append(value)

In [458]:
pd.DataFrame(fd)

,1 Goal,1 Shots,1 Shots on target,1 Shots off target,1 Blocked shots,1 Shots woodwork,1 Shots inside box,1 Shots outside box,1 Expected goals (xG),1 xG first half,...,2 Crosses,2 Accurate crosses,2 Throws,2 Yellow cards,2 Red cards,2 Saves,2 Diving saves,2 Saves inside box,2 Acted as sweeper,2 Punches
0,3,27,13,8,6,1,18,9,3.19,1.25,...,4,0,12,1,0,10,2,6,0,0


In [334]:
def getstats(url):
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    stats = driver.find_elements(By.CLASS_NAME, 'e360fsv1')[1].text
    game = driver.find_elements(By.CLASS_NAME, 'css-jkaqxa')[1].text
    time = driver.find_elements(By.CLASS_NAME, 'css-xj92le')[1].text
    driver.quit()
    if time != 'Half-Time':
        score = game.split()[::2]
        goal1 = score[0]
        goal2 = score[1]
        los = re.split(pattern = "\\r?\\n", string = stats)
        los.remove('What is xG?')
        los.remove('Keeper')
        los.remove('Discipline')
        columns = los[1::3]
        columns.insert(0, 'Goal')
        team1 = los[::3]
        team1.insert(0, goal1)
        team2 = los[2::3]
        team2.insert(0, goal2)
        cols = ['1 ' + word for word in columns] + ['2 ' + word for word in columns]
        gs = team1 + team2
        row = {k:v for k, v in zip(cols, gs)}
        return row
    
    

In [424]:
getstats("https://www.fotmob.com/match/3610143/matchfacts/liverpool-vs-brentford")

{'1 Goal': '3', '1 Shots': '27', '1 Shots on target': '13', '1 Shots off target': '8', '1 Blocked shots': '6', '1 Shots woodwork': '1', '1 Shots inside box': '18', '1 Shots outside box': '9', '1 Expected goals (xG)': '3.19', '1 xG first half': '1.25', '1 xG second half': '1.94', '1 xG on target (xGOT)': '3.72', '1 xG open play': '2.18', '1 xG set play': '1.01', '1 Duels won': '46', '1 Duels': '98', '1 Dribbles attempted': '20', '1 Dribbles succeeded': '9', '1 Tackles attempted': '9', '1 Tackles succeeded': '7', '1 Aerials won': '23', '1 Interceptions': '9', '1 Accurate passes': '574', '1 Own half': '206', '1 Opposition half': '368', '1 Passes': '694', '1 Pass success': '83%', '1 Touches': '883', '1 Long balls': '74', '1 Accurate long balls': '33', '1 Crosses': '32', '1 Accurate crosses': '4', '1 Throws': '26', '1 Yellow cards': '0', '1 Red cards': '0', '1 Saves': '1', '1 Diving saves': '1', '1 Saves inside box': '1', '1 Acted as sweeper': '1', '1 Punches': '0', '2 Goal': '0', '2 Shots'